In [2]:
import torch
import numpy as np
from torch import nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import matplotlib.pyplot as plt

In [3]:
states_in_the_prediction = ['01','02','04','05','06','08','09','10','12','13','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32','33','34','35','36','37','38','39','40',
 '41','42','44','45','46','47','48','49','50','51','53','54','55','56']

In [61]:
truth_df = pd.read_csv("../data/CDC/truth-Incident Hospitalizations 3.csv")
truth_df = truth_df[truth_df['date'] >= '2022-01-01']
#truth_df = truth_df[(truth_df['date'] >= '2022-01-01') & (truth_df['date'] <= '2023-01-28')] #truth_df = truth_df[truth_df['date'] >= '2022-01-01']
truth_df = truth_df[truth_df['date'] >= '2022-01-01']
truth_df = truth_df[truth_df['location'] != 'US']
truth_df = truth_df[truth_df['location'].isin(states_in_the_prediction)]
truth_df.sort_values(by=['date', 'location'], inplace=True)
unique_dates = truth_df['date'].unique()
unique_states = truth_df['location'].unique()

In [62]:
unique_dates

array(['2022-01-01', '2022-01-08', '2022-01-15', '2022-01-22',
       '2022-01-29', '2022-02-05', '2022-02-12', '2022-02-19',
       '2022-02-26', '2022-03-05', '2022-03-12', '2022-03-19',
       '2022-03-26', '2022-04-02', '2022-04-09', '2022-04-16',
       '2022-04-23', '2022-04-30', '2022-05-07', '2022-05-14',
       '2022-05-21', '2022-05-28', '2022-06-04', '2022-06-11',
       '2022-06-18', '2022-06-25', '2022-07-02', '2022-07-09',
       '2022-07-16', '2022-07-23', '2022-07-30', '2022-08-06',
       '2022-08-13', '2022-08-20', '2022-08-27', '2022-09-03',
       '2022-09-10', '2022-09-17', '2022-09-24', '2022-10-01',
       '2022-10-08', '2022-10-15', '2022-10-22', '2022-10-29',
       '2022-11-05', '2022-11-12', '2022-11-19', '2022-11-26',
       '2022-12-03', '2022-12-10', '2022-12-17', '2022-12-24',
       '2022-12-31', '2023-01-07', '2023-01-14', '2023-01-21',
       '2023-01-28', '2023-02-04', '2023-02-11', '2023-02-18',
       '2023-02-25'], dtype=object)

In [5]:
#### Standardize the data individually for each state
# mean_arr = []
# std_arr = []
# for state in unique_states:
#     state_values = truth_df[truth_df['location'] == state]['value'].values
#     mean = state_values.mean()
#     mean_arr.append(mean)
#     std = state_values.std()
#     std_arr.append(std)
#     truth_df.loc[truth_df['location'] == state, 'norm_value'] = (state_values - mean) / std

In [63]:
weeks = np.zeros([len(unique_dates),len(unique_states)])
for id1,i in enumerate(unique_dates):
    for id2,j in enumerate(unique_states):
        weeks[id1,id2] = truth_df[(truth_df['date']==i) & (truth_df['location']==j)]['value'].values
        # weeks[id1,id2] = truth_df[(truth_df['date']==i) & (truth_df['location']==j)]['norm_value'].values

In [64]:
weeks.shape

(61, 50)

In [21]:
# loader_temp = weeks.copy().transpose(1,0)
# loader_temp[0]
# loader_temp = weeks.copy().transpose(1,0)
# train_data = loader_temp[:,:-2].copy() #train_data = loader_temp[:,:-4].copy()
# np.concatenate([train_data[:,i:i+10] for i in range(0, 46, 1)], axis = 0)[50]

In [9]:
# split the data into train and validation sets
loader_temp = weeks.copy().transpose(1,0)
train_data = loader_temp[:,:-2].copy() #train_data = loader_temp[:,:-4].copy()
train_data = np.concatenate([train_data[:,i:i+10] for i in range(0, 46, 1)], axis = 0)
np.random.shuffle(train_data)
val_data = train_data[:500]
train_data = train_data[500:]
test_data = loader_temp[:,-8:].copy() # test_data = loader_temp[:,-10:].copy()

In [27]:
loader_temp[-1]

array([20., 10.,  2.,  9.,  1.,  1.,  2.,  2.,  5.,  5.,  4.,  4.,  6.,
        4.,  8.,  8.,  1.,  5.,  6.,  4.,  4.,  6.,  9.,  6.,  3.,  0.,
        2.,  2.,  2.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  3.,  1.,  5.,  8., 13., 47., 45., 73., 52., 32., 50.,
       50., 33., 16., 11.,  5.,  1.])

In [29]:
test_data[-1][-6:]

array([50., 33., 16., 11.,  5.,  1.])

#### Previous 6 weeks for the next 1 weeks

In [40]:
# convert the numpy arrays to PyTorch tensors
# train_inputs = torch.tensor(train_data[:, :6], dtype=torch.float32)
# train_labels = torch.tensor(train_data[:, 6:], dtype=torch.float32)

# val_inputs = torch.tensor(val_data[:, :6], dtype=torch.float32)
# val_labels = torch.tensor(val_data[:, 6:], dtype=torch.float32)

# test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
# test_labels = torch.tensor(test_data[:, 6:], dtype=torch.float32)

train_inputs = torch.tensor(train_data[:, :6], dtype=torch.float32)
train_labels = torch.tensor(train_data[:, 8:9], dtype=torch.float32) #train_labels = torch.tensor(train_data[:, 6:], dtype=torch.float32)

val_inputs = torch.tensor(val_data[:, :6], dtype=torch.float32)
val_labels = torch.tensor(val_data[:, 8:9], dtype=torch.float32) #val_labels = torch.tensor(val_data[:, 6:], dtype=torch.float32)

test_inputs = torch.tensor(test_data[:, -6:], dtype=torch.float32)
test_labels = torch.tensor(test_data[:, 6:7], dtype=torch.float32) # test_labels = torch.tensor(test_data[:, 6:], dtype=torch.float32)

mean = train_inputs.mean()
std = train_inputs.std()

train_inputs = (train_inputs - mean) / std
val_inputs = (val_inputs - mean) / std
train_labels = (train_labels - mean) / std
val_labels = (val_labels - mean) / std
test_inputs = (test_inputs  - mean) / std

In [41]:
# create the datasets
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
val_dataset =torch.utils.data.TensorDataset(val_inputs, val_labels)

# create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle = False)

In [12]:
# import pickle
# # save
# with open('../data/shuffled_data/MLP_flu_train.pickle', 'wb') as handle:
#     pickle.dump(train_dataset, handle)
# with open('../data/shuffled_data/MLP_flu_val.pickle', 'wb') as handle:
#     pickle.dump(val_dataset, handle)

In [13]:
train_loader.dataset.tensors[0].shape

torch.Size([1800, 6])

In [109]:
newtruth_df = pd.read_csv("../data/CDC/truth-Incident Hospitalizations 3.csv")
# truth_df = truth_df[(truth_df['date'] >= '2022-02-26') & (truth_df['date'] < '2023-01-28')]
newtruth_df = newtruth_df[newtruth_df['location'] != 'US']
newtruth_df = newtruth_df[newtruth_df['location'].isin(states_in_the_prediction)]
newtruth_df.sort_values(by=['date', 'location'], inplace=True)
groundtruth_0128 = np.array(newtruth_df[newtruth_df['date'] == '2023-01-28']['value'])
groundtruth_0204 = np.array(newtruth_df[newtruth_df['date'] == '2023-02-04']['value'])
groundtruth_0211 = np.array(newtruth_df[newtruth_df['date'] == '2023-02-11']['value'])
groundtruth_0218 = np.array(newtruth_df[newtruth_df['date'] == '2023-02-18']['value'])
groundtruth_0225 = np.array(newtruth_df[newtruth_df['date'] == '2023-02-25']['value'])

In [14]:
class AutoMLP(nn.Module):
    def __init__(self,input_length, output_length,hidden_length):
        super(AutoMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_length, hidden_length),
            nn.ReLU(),
            nn.Linear(hidden_length, hidden_length),
            nn.ReLU(),
            nn.Linear(hidden_length, output_length),
        )
    def forward(self, x):
        return self.model(x)

In [15]:
# model = AutoMLP(6, 1, 128).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01) #optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay

In [44]:
def week1():
    result_dict_1 = {}
    week_1_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_1_pred = np.abs(flat_list)
            week_1_pred_dict[(length,lrate)] = week_1_pred
            result_dict_1[(length,lrate)] = np.mean(np.abs(week_1_pred - groundtruth_0128))
        print(result_dict_1)
    return result_dict_1, week_1_pred_dict

In [34]:
temp_pred_1 = week_1_pred_dict.copy()
temp_result_1 = result_dict_1.copy()

In [36]:
result_dict_1

{(8, 0.01): 30.90858528137207,
 (8, 0.001): 38.28237796783447,
 (8, 0.0001): 35.79594268798828,
 (16, 0.01): 21.833219985961914,
 (16, 0.001): 21.186963806152345,
 (16, 0.0001): 19.860978775024414,
 (32, 0.01): 125.61238632202148,
 (32, 0.001): 37.04625598907471,
 (32, 0.0001): 17.256411781311034,
 (64, 0.01): 31.7813041305542,
 (64, 0.001): 21.980377502441407,
 (64, 0.0001): 25.619525413513184,
 (128, 0.01): 21.802515449523927,
 (128, 0.001): 18.710785522460938,
 (128, 0.0001): 23.489974174499512,
 (256, 0.01): 38.044397811889645,
 (256, 0.001): 24.890918807983397,
 (256, 0.0001): 17.519793395996093}

In [18]:
result_dict_1

{(8, 0.01): 23.575529823303224,
 (8, 0.001): 21.975960998535157,
 (8, 0.0001): 19.39921875,
 (16, 0.01): 25.01174301147461,
 (16, 0.001): 11.461907119750977,
 (16, 0.0001): 40.021451454162595,
 (32, 0.01): 17.59495834350586,
 (32, 0.001): 37.26073196411133,
 (32, 0.0001): 22.543455963134765,
 (64, 0.01): 15.459693565368653,
 (64, 0.001): 29.50738105773926,
 (64, 0.0001): 14.658164291381835,
 (128, 0.01): 12.145687408447266,
 (128, 0.001): 24.456424598693847,
 (128, 0.0001): 15.56952480316162,
 (256, 0.01): 15.825345840454101,
 (256, 0.001): 15.539810409545899,
 (256, 0.0001): 16.675416831970214}

In [45]:
def week2():
    result_dict_2 = {}
    week_2_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_2_pred = np.abs(flat_list)
            week_2_pred_dict[(length,lrate)] = week_2_pred
            result_dict_2[(length,lrate)] = np.mean(np.abs(week_2_pred - groundtruth_0204))
        print(result_dict_2)
    return result_dict_2, week_2_pred_dict

In [48]:
def week3():
    result_dict_3 = {}
    week_3_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_3_pred = np.abs(flat_list)
            week_3_pred_dict[(length,lrate)] = week_3_pred
            result_dict_3[(length,lrate)] = np.mean(np.abs(week_3_pred - groundtruth_0211))
        print(result_dict_3)
    return result_dict_3, week_3_pred_dict

In [47]:
def week4():
    result_dict_4 = {}
    week_4_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_4_pred = np.abs(flat_list)
            week_4_pred_dict[(length,lrate)] = week_4_pred
            result_dict_4[(length,lrate)] = np.mean(np.abs(week_4_pred - groundtruth_0204))
        print(result_dict_4)
    return result_dict_4, week_4_pred_dict

In [131]:
result_dict_2 = result_dict

In [139]:
result_dict_4

{(8, 0.01): 31.279174613952637,
 (8, 0.001): 34.62915599822998,
 (8, 0.0001): 84.24005493164063,
 (16, 0.01): 28.474387435913087,
 (16, 0.001): 25.021390647888182,
 (16, 0.0001): 33.25743797302246,
 (32, 0.01): 23.506855926513673,
 (32, 0.001): 26.89193504333496,
 (32, 0.0001): 28.327368278503418,
 (64, 0.01): 22.06661148071289,
 (64, 0.001): 27.043925132751465,
 (64, 0.0001): 20.53235019683838,
 (128, 0.01): 31.234378204345703,
 (128, 0.001): 23.605826377868652,
 (128, 0.0001): 24.16839130401611,
 (256, 0.01): 21.985442123413087,
 (256, 0.001): 23.902833671569823,
 (256, 0.0001): 62.7376868057251}

In [128]:
result_dict_3

{(8, 0.01): 33.38484477996826,
 (8, 0.001): 38.13541572570801,
 (8, 0.0001): 29.91273036956787,
 (16, 0.01): 23.280872650146485,
 (16, 0.001): 33.50578121185303,
 (16, 0.0001): 32.05569496154785,
 (32, 0.01): 18.157356834411623,
 (32, 0.001): 25.96607925415039,
 (32, 0.0001): 33.00566864013672,
 (64, 0.01): 30.06999069213867,
 (64, 0.001): 43.26408473968506,
 (64, 0.0001): 16.76365566253662,
 (128, 0.01): 52.16321346282959,
 (128, 0.001): 23.476780471801757,
 (128, 0.0001): 29.579945182800294,
 (256, 0.01): 65.37937660217285,
 (256, 0.001): 22.718031806945802,
 (256, 0.0001): 52.36312831878662}

In [165]:
result_dict_2

{(8, 0.01): 36.262850799560546,
 (8, 0.001): 20.73842296600342,
 (8, 0.0001): 103.26102088928222,
 (16, 0.01): 36.05392253875733,
 (16, 0.001): 25.70744213104248,
 (16, 0.0001): 27.64268928527832,
 (32, 0.01): 27.38308036804199,
 (32, 0.001): 25.030885543823242,
 (32, 0.0001): 32.5325424194336,
 (64, 0.01): 48.99851676940918,
 (64, 0.001): 31.491930809020996,
 (64, 0.0001): 20.152099800109863,
 (128, 0.01): 32.94138061523437,
 (128, 0.001): 29.512160263061524,
 (128, 0.0001): 28.101052551269532,
 (256, 0.01): 30.134797286987304,
 (256, 0.001): 21.791844749450682,
 (256, 0.0001): 27.589387321472167}

In [186]:
(13.26+20+15+17)/4

16.314999999999998

In [171]:
# load in the updated truth 
new_truth = pd.read_csv("../data/CDC/truth-Incident Hospitalizations 2.csv")
new_truth = new_truth[new_truth['location'] != 'US']
new_truth = new_truth[new_truth['location'].isin(states_in_the_prediction)]
new_truth.sort_values(by=['date', 'location'], inplace=True)
groundtruth_0128 = np.array(new_truth[new_truth['date'] == '2023-01-28']['value'])
groundtruth_0204 = np.array(new_truth[new_truth['date'] == '2023-02-04']['value'])
groundtruth_0211 = np.array(new_truth[new_truth['date'] == '2023-02-11']['value'])
groundtruth_0218 = np.array(new_truth[new_truth['date'] == '2023-02-18']['value'])

In [183]:
for item in week_3_pred_dict.items():
    print(item[0], np.mean(np.abs(item[1] - groundtruth_0211)))

(8, 0.01) 26.53727779388428
(8, 0.001) 42.19153205871582
(8, 0.0001) 30.798382225036622
(16, 0.01) 26.973972930908204
(16, 0.001) 34.70088481903076
(16, 0.0001) 22.362245864868164
(32, 0.01) 19.21786144256592
(32, 0.001) 20.663228912353517
(32, 0.0001) 35.254453887939455
(64, 0.01) 27.425651397705078
(64, 0.001) 54.687352867126464
(64, 0.0001) 20.334604606628417
(128, 0.01) 62.8641361618042
(128, 0.001) 15.658547439575194
(128, 0.0001) 38.08612743377685
(256, 0.01) 75.00898101806641
(256, 0.001) 17.497203788757325
(256, 0.0001) 63.88647518157959


In [182]:
for item in week_4_pred_dict.items():
    print(item[0], np.mean(np.abs(item[1] - groundtruth_0218)))

(8, 0.01) 21.0478023147583
(8, 0.001) 40.64174816131592
(8, 0.0001) 88.46328689575195
(16, 0.01) 26.694155349731446
(16, 0.001) 19.77414150238037
(16, 0.0001) 30.32465919494629
(32, 0.01) 24.555188293457032
(32, 0.001) 32.745373306274416
(32, 0.0001) 21.276368827819823
(64, 0.01) 18.176981811523437
(64, 0.001) 23.638693962097168
(64, 0.0001) 17.372859001159668
(128, 0.01) 35.10382652282715
(128, 0.001) 21.371487464904785
(128, 0.0001) 24.67693058013916
(256, 0.01) 21.69838691711426
(256, 0.001) 23.185288505554198
(256, 0.0001) 67.91990558624268


In [185]:
GLEAM_01_23_pd = pd.read_csv("../data/GLEAM/2023-01-23-MOBS-GLEAM_FLUH.csv")
GLEAM_01_28_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '1 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
GLEAM_02_04_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '2 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
GLEAM_02_11_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '3 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
GLEAM_02_18_pred = GLEAM_01_23_pd[(GLEAM_01_23_pd['target'] == '4 wk ahead inc flu hosp') & (GLEAM_01_23_pd['quantile'] == 0.5) & (GLEAM_01_23_pd['location'].isin(states_in_the_prediction))][['location','value']]['value'].to_numpy()
print('GLEAM 1 week ahead: ',np.mean(np.abs(GLEAM_01_28_pred - groundtruth_0128)))
print('GLEAM 2 week ahead: ',np.mean(np.abs(GLEAM_02_04_pred - groundtruth_0204)))
print('GLEAM 3 week ahead: ',np.mean(np.abs(GLEAM_02_11_pred - groundtruth_0211)))
print('GLEAM 4 week ahead: ',np.mean(np.abs(GLEAM_02_18_pred - groundtruth_0218)))
print('Average GLEAM: ',np.mean([np.mean(np.abs(GLEAM_01_28_pred - groundtruth_0128)),np.mean(np.abs(GLEAM_02_04_pred - groundtruth_0204)),np.mean(np.abs(GLEAM_02_11_pred - groundtruth_0211)),np.mean(np.abs(GLEAM_02_18_pred - groundtruth_0218))]))

GLEAM 1 week ahead:  18.702641530396498
GLEAM 2 week ahead:  12.972103543433212
GLEAM 3 week ahead:  12.099102270072143
GLEAM 4 week ahead:  12.208814474872613
Average GLEAM:  13.995665454693617


In [188]:
groundtruth_0218

array([ 29,   5,  35,  24,  80,  17,  16,   1, 216,  44,   4,   8,  26,
        35,   9,   4,  20,  28,   5,  25,  30,  57,   8,  30,  45,   3,
         5,  19,   4,  57,   3,  86,  43,  10,  40,  42,   4, 188,   2,
        42,  10,  48, 232,   4,   2,  54,  40,  10,   5,   1])

16: 1 week ahead:  19.93516399383545 lr=0.0001; 16: 1 week ahead:  19.870430908203126 lr = 0.01; <br>
64: 1 week ahead:  21.80642028808594 lr=0.0001; 64: 1 week ahead:  17.2224405670166 lr = 0.01; <br>
128: 1 week ahead:  18.330050354003905 lr=0.0001; 128: 1 week ahead:  14.73071044921875 lr = 0.01; <br>
256: 2 week ahead:  22.322580184936523 lr=0.001; 256: 2 week ahead:   lr = 0.01; <br>

In [20]:
# tryput_1 = (best_model(test_inputs.to(device))* std + mean)
# np.mean(np.abs(np.abs([item for sublist in tryput_1.detach().numpy() for item in sublist]) - np.array(truth_df[truth_df['date'] == '2023-01-28']['value'])))

In [94]:
# pred = best_model(test_inputs.to(device))[:,np.array([False, False, False,True,])]
# result = []
# for i in range(len(pred)):
#     result.append(((pred[i] * std_arr[i])+ mean_arr[i]).tolist())
# flat_list = [item for sublist in result for item in sublist]
# p.mean(np.abs(flat_list - np.array(truth_df[truth_df['date'] == '2023-02-04']['value'])))

In [103]:
test_pred_1 = best_model(test_inputs.to(device)) * std + mean
flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
week_1_pred = np.abs(flat_list)
print('1 week ahead: ',np.mean(np.abs(week_1_pred - groundtruth_0204)))

1 week ahead:  78.5850626373291


In [13]:
test_preds = best_model(test_inputs.to(device)) * std + mean
# test_preds = best_model(test_inputs.to(device)) * std + mean
# print("test error:", torch.mean(torch.abs(test_preds.cpu() - test_labels)))

temp = test_preds[:,np.array([True, False, False,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_1_pred = np.abs(flat_list)

temp = test_preds[:,np.array([False, True, False,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_2_pred = np.abs(flat_list)

temp = test_preds[:,np.array([False, False, True,False,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_3_pred = np.abs(flat_list)


temp = test_preds[:,np.array([False, False, False,True,])]
flat_list = [item for sublist in temp.detach().numpy() for item in sublist]
week_4_pred = np.abs(flat_list)

In [15]:
np.stack([groundtruth_0128,groundtruth_0204])[:,0]

array([42, 34])

In [16]:
print('1 week ahead: ',np.mean(np.abs(week_1_pred - groundtruth_0128)))
print('2 week ahead: ',np.mean(np.abs(week_2_pred - groundtruth_0204)))
print('3 week ahead: ',np.mean(np.abs(week_3_pred - groundtruth_0128)))
print('4 week ahead: ',np.mean(np.abs(week_4_pred - groundtruth_0204)))

1 week ahead:  21.5026469039917
2 week ahead:  22.64774673461914
3 week ahead:  29.41070137023926
4 week ahead:  43.41663303375244


In [144]:
# proportions = [.90, .10]
# lengths = [int(p * len(train_loader)) for p in proportions]
# lengths[-1] = len(train_loader) - sum(lengths[:-1])
# tr_dataset, vl_dataset = torch.utils.data.random_split(train_loader, lengths)

In [ ]:
# class AutoMLP(nn.Module):
#     def __init__(self,input_length, hidden_length, output_length):
#         super(AutoMLP, self).__init__()
#         self.input_length = input_length
#         self.hidden_length = hidden_length
#         self.output_length = output_length
#         self.fc1 = nn.Linear(self.input_length, self.hidden_length)
#         self.fc2 = nn.Linear(self.hidden_length, self.output_length)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)

In [ ]:
def get_plots_quantile_prediction(npz_file_location):
    npz = np.load(npz_file_location)
    pred = npz['prediction']
    
    # truth = npz['truth']
    ground_truth = np.resize(dfg_date_filtered_truth_filtered[dfg_date_filtered_truth_filtered['date'] == '2023-01-21']['value'].to_numpy(), (4,50)).T

    # print(pred.shape,truth.shape)
    # if "quantile_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4 ,3)
    #     truth_reshaped = truth.reshape(50, 4)
    # elif "dropout_model" in npz_file_location:
    #     pred_reshaped = pred.reshape(50, 4)
    #     truth_reshaped = truth.reshape(50, 4)
    pred_reshaped = pred.reshape(50, 4 ,3)
    pred_actual_ground_truth = np.resize(dfg_date_filtered_truth_filtered[dfg_date_filtered_truth_filtered['date'].isin(pd.date_range('2023-01-21', '2023-01-28', freq='W-SAT'))]['value'].to_numpy(), (4,50)).T
    # truth_reshaped = truth.reshape(50, 4)

    x = np.arange(1,5)
    # states = list(states_data[1])
    states = dfg_date_filtered_truth_filtered['location_name'].unique()
    # states = states_in_the_prediction
    for i in range(50):
        plt.plot(x, pred_reshaped[i,:,1] + ground_truth[i,:],label="Prediction", marker = "*")
        plt.plot(x, pred_reshaped[i,:,0]  + ground_truth[i,:],label="Lower Bound", marker = "v")
        plt.plot(x, pred_reshaped[i,:,2]  + ground_truth[i,:],label="Upper Bound", marker = "^")
        plt.plot(x, ground_truth[i,:],label="Truth Used for Prediction", marker = ".")
        plt.plot(np.arange(1,3), pred_actual_ground_truth[i,:2],label="Truth", marker = "o")
        plt.title(states[i])
        plt.xlabel("Num of Weeks Ahead")
        plt.ylabel("Number of Flu Cases")
        plt.legend(loc='best')
        # plt.fill_between(x, pred_reshaped[i,:,0] + ground_truth[i,:],pred_reshaped[i,:,2] + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.fill_between(x, pred_reshaped[i,:,0]  + ground_truth[i,:],pred_reshaped[i,:,2]  + ground_truth[i,:],color = 'green',alpha = 0.2)
        plt.show()

#### 2023 Feb 27

In [83]:
# split the data into train and validation sets
loader_temp = weeks.copy().transpose(1,0)
train_data = loader_temp[:,:-4].copy() #train_data = loader_temp[:,:-4].copy()
train_data = np.concatenate([train_data[:,i:i+10] for i in range(0, 48, 1)], axis = 0)
np.random.shuffle(train_data)
val_data = train_data[:700]
train_data = train_data[700:]
test_data = loader_temp[:,-10:].copy() # test_data = loader_temp[:,-10:].copy()

In [125]:
train_inputs = torch.tensor(train_data[:, :6], dtype=torch.float32)
train_labels = torch.tensor(train_data[:, 9:10], dtype=torch.float32) #train_labels = torch.tensor(train_data[:, 6:], dtype=torch.float32)

val_inputs = torch.tensor(val_data[:, :6], dtype=torch.float32)
val_labels = torch.tensor(val_data[:, 9:10], dtype=torch.float32) #val_labels = torch.tensor(val_data[:, 6:], dtype=torch.float32)

test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_labels = torch.tensor(test_data[:, 8:9], dtype=torch.float32) # test_labels = torch.tensor(test_data[:, 6:], dtype=torch.float32)

mean = train_inputs.mean()
std = train_inputs.std()

train_inputs = (train_inputs - mean) / std
val_inputs = (val_inputs - mean) / std
train_labels = (train_labels - mean) / std
val_labels = (val_labels - mean) / std
test_inputs = (test_inputs  - mean) / std

# create the datasets
train_dataset = torch.utils.data.TensorDataset(train_inputs, train_labels)
val_dataset =torch.utils.data.TensorDataset(val_inputs, val_labels)

# create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=8, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle = False)

In [94]:
def week1():
    result_dict_1 = {}
    week_1_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_1_pred = np.abs(flat_list)
            week_1_pred_dict[(length,lrate)] = week_1_pred
            result_dict_1[(length,lrate)] = np.mean(np.abs(week_1_pred - groundtruth_0204))
        print(result_dict_1)
    return result_dict_1, week_1_pred_dict

In [100]:
def week2():
    result_dict_2 = {}
    week_2_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_2_pred = np.abs(flat_list)
            week_2_pred_dict[(length,lrate)] = week_2_pred
            result_dict_2[(length,lrate)] = np.mean(np.abs(week_2_pred - groundtruth_0211))
        print(result_dict_2)
    return result_dict_2, week_2_pred_dict

In [107]:
def week3():
    result_dict_3 = {}
    week_3_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_3_pred = np.abs(flat_list)
            week_3_pred_dict[(length,lrate)] = week_3_pred
            result_dict_3[(length,lrate)] = np.mean(np.abs(week_3_pred - groundtruth_0218))
        print(result_dict_3)
    return result_dict_3, week_3_pred_dict

In [110]:
def week4():
    result_dict_4 = {}
    week_4_pred_dict = {}
    for length in [8,16,32,64,128,256]:
        for lrate in [0.01,0.001,0.0001]:
            model = AutoMLP(6, 1, length).to(device) # 8, 16, 32, 64, 128, 256 model = AutoMLP(6, 4, 256).to(device) # 8, 16, 32, 64, 128, 256
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lrate) #optimizer = optim.Adam(model.parameters(), lr=0.001)
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 10, gamma=0.9) # stepwise learning rate decay
            best_val = 1e6
            for epoch in range(300):
                running_loss = []
                for i, data in enumerate(train_loader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss.append(loss.item())
                # validate the model after each epoch
                model.eval()
                with torch.no_grad():
                    val_running_loss = []
                    for i, data in enumerate(val_loader, 0):
                        inputs, labels = data
                        inputs, labels = inputs.to(device), labels.to(device)
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        val_running_loss.append(loss.item())
                if epoch % 10 == 0:
                    print('Epoch %d Training loss: %.3f Validation loss : %.3f' % (epoch + 1, np.mean(running_loss),  np.mean(val_running_loss)))
                scheduler.step()
                if np.mean(val_running_loss) < best_val:
                    best_val = np.mean(val_running_loss)
                    best_model = model

            test_pred_1 = best_model(test_inputs.to(device)) * std + mean
            flat_list = [item for sublist in test_pred_1.detach().numpy() for item in sublist]
            week_4_pred = np.abs(flat_list)
            week_4_pred_dict[(length,lrate)] = week_4_pred
            result_dict_4[(length,lrate)] = np.mean(np.abs(week_4_pred - groundtruth_0225))
        print(result_dict_4)
    return result_dict_4, week_4_pred_dict

In [95]:
week_1_pred_2_04 = week1()

Epoch 1 Training loss: 0.480 Validation loss : 0.702
Epoch 11 Training loss: 0.169 Validation loss : 0.076
Epoch 21 Training loss: 0.130 Validation loss : 0.108
Epoch 31 Training loss: 0.126 Validation loss : 0.099
Epoch 41 Training loss: 0.140 Validation loss : 0.069
Epoch 51 Training loss: 0.104 Validation loss : 0.069
Epoch 61 Training loss: 0.106 Validation loss : 0.101
Epoch 71 Training loss: 0.110 Validation loss : 0.087
Epoch 81 Training loss: 0.112 Validation loss : 0.071
Epoch 91 Training loss: 0.101 Validation loss : 0.065
Epoch 101 Training loss: 0.096 Validation loss : 0.071
Epoch 111 Training loss: 0.098 Validation loss : 0.095
Epoch 121 Training loss: 0.093 Validation loss : 0.075
Epoch 131 Training loss: 0.095 Validation loss : 0.072
Epoch 141 Training loss: 0.088 Validation loss : 0.090
Epoch 151 Training loss: 0.085 Validation loss : 0.138
Epoch 161 Training loss: 0.087 Validation loss : 0.102
Epoch 171 Training loss: 0.086 Validation loss : 0.074
Epoch 181 Training lo

In [96]:
week_1_pred_2_04[0]

{(8, 0.01): 22.205411987304686,
 (8, 0.001): 27.826680297851563,
 (8, 0.0001): 42.29873199462891,
 (16, 0.01): 20.938849716186525,
 (16, 0.001): 15.04667465209961,
 (16, 0.0001): 38.37846633911133,
 (32, 0.01): 15.871765670776368,
 (32, 0.001): 15.222069473266602,
 (32, 0.0001): 20.545441513061522,
 (64, 0.01): 14.731124267578124,
 (64, 0.001): 18.19775146484375,
 (64, 0.0001): 16.865403289794923,
 (128, 0.01): 15.488911972045898,
 (128, 0.001): 15.631195831298829,
 (128, 0.0001): 10.902928085327149,
 (256, 0.01): 22.282574234008788,
 (256, 0.001): 12.839627990722656,
 (256, 0.0001): 18.399097290039062}

In [138]:
week1_mae = week_1_pred_2_04[0][(128, 0.0001)]

In [98]:
week1_mlp_pred = week_1_pred_2_04[1][(128, 0.0001)]

In [101]:
week_2_pred_2_11 = week2()

Epoch 1 Training loss: 0.569 Validation loss : 0.480
Epoch 11 Training loss: 0.345 Validation loss : 0.422
Epoch 21 Training loss: 0.316 Validation loss : 0.348
Epoch 31 Training loss: 0.320 Validation loss : 0.417
Epoch 41 Training loss: 0.318 Validation loss : 0.370
Epoch 51 Training loss: 0.301 Validation loss : 0.326
Epoch 61 Training loss: 0.298 Validation loss : 0.375
Epoch 71 Training loss: 0.293 Validation loss : 0.338
Epoch 81 Training loss: 0.287 Validation loss : 0.339
Epoch 91 Training loss: 0.281 Validation loss : 0.339
Epoch 101 Training loss: 0.278 Validation loss : 0.339
Epoch 111 Training loss: 0.286 Validation loss : 0.342
Epoch 121 Training loss: 0.275 Validation loss : 0.363
Epoch 131 Training loss: 0.273 Validation loss : 0.345
Epoch 141 Training loss: 0.276 Validation loss : 0.332
Epoch 151 Training loss: 0.265 Validation loss : 0.348
Epoch 161 Training loss: 0.263 Validation loss : 0.347
Epoch 171 Training loss: 0.260 Validation loss : 0.352
Epoch 181 Training lo

In [137]:
week2_mae = week_2_pred_2_11[0][(256, 0.0001)]

In [103]:
week2_mlp_pred = week_1_pred_2_04[1][(256, 0.0001)]

In [108]:
wee3_pred_2_18 = week3()

Epoch 1 Training loss: 0.801 Validation loss : 0.735
Epoch 11 Training loss: 0.544 Validation loss : 0.614
Epoch 21 Training loss: 0.512 Validation loss : 0.601
Epoch 31 Training loss: 0.474 Validation loss : 0.618
Epoch 41 Training loss: 0.481 Validation loss : 0.573
Epoch 51 Training loss: 0.484 Validation loss : 0.603
Epoch 61 Training loss: 0.482 Validation loss : 0.579
Epoch 71 Training loss: 0.470 Validation loss : 0.554
Epoch 81 Training loss: 0.477 Validation loss : 0.585
Epoch 91 Training loss: 0.466 Validation loss : 0.596
Epoch 101 Training loss: 0.463 Validation loss : 0.556
Epoch 111 Training loss: 0.453 Validation loss : 0.551
Epoch 121 Training loss: 0.453 Validation loss : 0.569
Epoch 131 Training loss: 0.454 Validation loss : 0.584
Epoch 141 Training loss: 0.455 Validation loss : 0.578
Epoch 151 Training loss: 0.452 Validation loss : 0.561
Epoch 161 Training loss: 0.452 Validation loss : 0.559
Epoch 171 Training loss: 0.448 Validation loss : 0.567
Epoch 181 Training lo

In [122]:
week3_mae = wee3_pred_2_18[0][(256, 0.0001)]

In [118]:
week3_mlp_pred = wee3_pred_2_18[1][(256, 0.0001)]

In [126]:
week4_pred_2_26 = week4()

Epoch 1 Training loss: 1.148 Validation loss : 0.691
Epoch 11 Training loss: 0.801 Validation loss : 0.608
Epoch 21 Training loss: 0.771 Validation loss : 0.589
Epoch 31 Training loss: 0.755 Validation loss : 0.612
Epoch 41 Training loss: 0.748 Validation loss : 0.590
Epoch 51 Training loss: 0.701 Validation loss : 0.589
Epoch 61 Training loss: 0.693 Validation loss : 0.612
Epoch 71 Training loss: 0.690 Validation loss : 0.606
Epoch 81 Training loss: 0.620 Validation loss : 0.641
Epoch 91 Training loss: 0.606 Validation loss : 0.636
Epoch 101 Training loss: 0.584 Validation loss : 0.757
Epoch 111 Training loss: 0.557 Validation loss : 0.852
Epoch 121 Training loss: 0.551 Validation loss : 0.694
Epoch 131 Training loss: 0.548 Validation loss : 0.592
Epoch 141 Training loss: 0.536 Validation loss : 0.626
Epoch 151 Training loss: 0.506 Validation loss : 0.598
Epoch 161 Training loss: 0.509 Validation loss : 0.600
Epoch 171 Training loss: 0.497 Validation loss : 0.615
Epoch 181 Training lo

In [127]:
week4_mae = week4_pred_2_26[0][(256, 0.0001)]

In [128]:
week4_mlp_pred = week4_pred_2_26[1][(256, 0.0001)]

In [141]:
print(week1_mae, week2_mae, week3_mae, week4_mae)
mae_all_four_weeks = np.mean([week1_mae, week2_mae, week3_mae, week4_mae])
mae_all_four_weeks

10.902928085327149 17.37345802307129 15.713558578491211 15.227695083618164


14.804409942626954

In [135]:
mlp_all_four_weeks = np.stack([week1_mlp_pred, week2_mlp_pred, week3_mlp_pred, week4_mlp_pred], axis=0)

In [136]:
mlp_all_four_weeks

array([[4.28645134e+01, 2.09407043e+00, 5.44606361e+01, 2.75767860e+01,
        9.86205063e+01, 1.86086960e+01, 1.44608574e+01, 2.04736328e+00,
        3.32840454e+02, 6.15409851e+01, 9.87140656e+00, 1.33667717e+01,
        5.42458992e+01, 6.34236488e+01, 2.81825638e+01, 1.74767723e+01,
        1.47852211e+01, 4.44486694e+01, 1.05986633e+01, 3.14857330e+01,
        6.62478104e+01, 8.57927399e+01, 1.51034775e+01, 2.97746010e+01,
        6.15858116e+01, 5.77153778e+00, 2.63474922e+01, 6.48354340e+00,
        1.67643623e+01, 6.76163559e+01, 1.01438179e+01, 1.27382507e+02,
        2.30765533e+01, 1.87202301e+01, 6.66986847e+01, 9.00839462e+01,
        1.09734840e+01, 1.83136826e+02, 4.34403992e+00, 3.80143852e+01,
        8.54008484e+00, 3.52451782e+01, 2.70767609e+02, 6.35208130e-01,
        2.45851898e+00, 4.12973480e+01, 4.47575302e+01, 1.08207664e+01,
        5.56143494e+01, 3.27552795e-01],
       [3.13779411e+01, 8.66275787e+00, 1.60542908e+01, 2.42709770e+01,
        1.22551750e+02,